In [309]:
def parse(input):
  x = -1
  lines = [l.strip() for l in open(input, 'r').readlines()]
  for y in range(len(lines)):
    x = lines[y].find('S')
    if x != -1:
      start = (x, y)
  return start, lines

parse('sample')

((1, 1), ['-L|F7', '7S-7|', 'L|7||', '-L-J|', 'L|-JF'])

In [310]:
def get_next(loop, pos):
  x, y = pos
  if loop[y][x] == '|':
    yield (x, y - 1)
    yield (x, y + 1)
  elif loop[y][x] == '-':
    yield (x - 1, y)
    yield (x + 1, y)
  elif loop[y][x] == 'L':
    yield (x, y - 1)
    yield (x + 1, y)
  elif loop[y][x] == 'J':
    yield (x, y - 1)
    yield (x - 1, y)
  elif loop[y][x] == '7':
    yield (x, y + 1)
    yield (x - 1, y)
  elif loop[y][x] == 'F':
    yield (x, y + 1)
    yield (x + 1, y)
  elif loop[y][x] == 'S':
    if y > 0 and loop[y - 1][x] in ['|', '7', 'F']:
      yield (x, y - 1)
    if y < len(loop) - 1 and loop[y + 1][x] in ['|', 'L', 'J']:
      yield (x, y + 1)
    if x > 0 and loop[y][x - 1] in ['-', 'F', 'L']:
      yield (x - 1, y)
    if x < len(loop[y]) - 1 and loop[y][x + 1] in ['-', '7', 'J']:
      yield (x + 1, y)

def first_part(input):
  start, loop = input
  steps = 0
  queue = [start]
  visited = set(start)

  while queue:
    next_queue = []
    for pos in queue:
      for n in get_next(loop, pos):
        if n not in visited:
          next_queue.append(n)
          visited.add(n)
    queue = next_queue
    steps += 1
  
  return steps - 1

assert first_part(parse('sample')) == 4
assert first_part(parse('sample2')) == 8
print(first_part(parse('input')))

6778


In [311]:
def get_direction(current, next):
  cx, cy = current
  nx, ny = next
  if cx == nx:
    return 'N' if cy > ny else 'S'
  else:
    return 'W' if cx > nx else 'E'

rotations = {
  'NW': -1,
  'NE': 1,
  'SW': 1,
  'SE': -1,
  'WN': 1,
  'WS': -1,
  'EN': -1,
  'ES': 1,
}

def get_to_the_right(current, next):
  cx, cy = current
  nx, ny = next
  dx = nx - cx
  dy = ny - cy
  yield (cx - dy, cy + dx)
  yield (cx + dx - dy, cy + dx + dy)

def get_neighbors(current):
  cx, cy = current
  yield (cx + 1, cy)
  yield (cx - 1, cy)
  yield (cx, cy + 1)
  yield (cx, cy - 1)

def second_part(input):
  current, loop = input
  path = [current]
  next = [next for next in get_next(loop, current)][0]
  start_direction = direction = get_direction(current, next)
  rotation = 0

  while True:
    current = next
    path.append(current)
    possible_next = [next for next in get_next(loop, current) if next not in path]
    if len(possible_next) == 0:
      break
    next = possible_next[0]
    next_direction = get_direction(current, next)
    rotation += rotations.get(direction + next_direction, 0)
    direction = next_direction
  next = path[0]
  next_direction = get_direction(current, next)
  rotation += rotations.get(direction + next_direction, 0)
  rotation += rotations.get(next_direction + start_direction, 0)

  path.append(path[0])
  if rotation == -4:
    path = list(reversed(path))

  inside = set()
  for current, next in zip(path, path[1:]):
    for neighbor in get_to_the_right(current, next):
      if neighbor not in path:
        inside.add(neighbor)
  
  queue = list(inside)
  while queue:
    next_queue = []
    for pos in queue:
      for n in get_neighbors(pos):
        if n not in inside and n not in path:
          next_queue.append(n)
          inside.add(n)
    queue = next_queue

  return len(inside)

assert second_part(parse('sample3')) == 4
assert second_part(parse('sample4')) == 8
assert second_part(parse('sample5')) == 10
print(second_part(parse('input')))

433
